**Engy-5310** Computational Continuum Transport Phenomena, UMass Lowell, Chem. Eng. Nuclear Program Spring 2021; Prof. V. F. de Almeida

# A Diffusion Drift-Flux Model for Two-Phase Mixture Flow
$  
  \newcommand{\Amtrx}{\boldsymbol{\mathsf{A}}}
  \newcommand{\Bmtrx}{\boldsymbol{\mathsf{B}}}
  \newcommand{\Mmtrx}{\boldsymbol{\mathsf{M}}}
  \newcommand{\Imtrx}{\boldsymbol{\mathsf{I}}}
  \newcommand{\Pmtrx}{\boldsymbol{\mathsf{P}}}
  \newcommand{\Lmtrx}{\boldsymbol{\mathsf{L}}}
  \newcommand{\Umtrx}{\boldsymbol{\mathsf{U}}}
  \newcommand{\Smtrx}{\boldsymbol{\mathsf{S}}}
  \newcommand{\xvec}{\boldsymbol{\mathsf{x}}}
  \newcommand{\avec}{\boldsymbol{\mathsf{a}}}
  \newcommand{\bvec}{\boldsymbol{\mathsf{b}}}
  \newcommand{\cvec}{\boldsymbol{\mathsf{c}}}
  \newcommand{\rvec}{\boldsymbol{\mathsf{r}}}
  \newcommand{\fvec}{\boldsymbol{\mathsf{f}}}
  \newcommand{\mvec}{\boldsymbol{\mathsf{m}}}
  \newcommand{\gvec}{\boldsymbol{\mathsf{g}}}
  \newcommand{\zerovec}{\boldsymbol{\mathsf{0}}}
  \newcommand{\norm}[1]{\bigl\lVert{#1}\bigr\rVert}
  \newcommand{\transpose}[1]{{#1}^\top}
  \DeclareMathOperator{\rank}{rank}
  \newcommand{\Reals}{\mathbb{R}}
  \newcommand{\thetavec}{\boldsymbol{\theta}}
$

**Student(s):**  <br>
**Course:** Engy-5310 Computational Continuum Transport Phenomena  <br>
**Mentor:** 

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**27Mar2021**

---
## Table of Contents<a id="toc"></a>
* [Introduction](#intro)
* [Problem Statement](#problem)
* [Objectives](#obj)
* [Approach](#app)
* [Results and Analysis](#ranalysis)
* [Conclusions](#conclusions)
* [References](#refs)
---

## [Introduction](#toc)<a id="intro"></a>

- The volume-average method is used for deriving a two-phase mixture balance equation for the motion of two fluid phases. The balance equation for the vapor phase is closed by a diffusion drift-flux constitutive equation. Boundary conditions for a developing fluid flow are used in cylindrical coordinates. This model is a prototype of a mixture theory for two-phase fluid flow in various equipments for example, steam generation from liquid feed water.

- Text... 

## [Problem Statement](#toc)<a id="problem"></a>

Solve the convective-diffusive model problem for a two-phase, 1-D mixture flow with phase change in a pipe. Compute the mixture velocity $v:[a,b]\subset\Reals\rightarrow\Reals$ and vapor fraction $\alpha:[a,b]\subset\Reals\rightarrow\Reals$ such that:

\begin{align*}
 \text{d}_z\bigl(\rho\,v\bigr) &= 0 \quad \forall \quad z\in [a,b] \\
 \text{d}_z\bigl(\alpha\rho_v\,v\bigr) - \text{d}_z\Bigl(\rho_\ell\,D\,\text{d}_z\bigl(\frac{\rho_v}{\alpha\rho_v + (1-\alpha)\rho_l}\bigr)\Bigr) - S_v & = 0 \quad \forall \quad z\in [a,b] \\
 v(a) &= v_0 \\
 \alpha(a) &= 0, \\
 -\rho\,D \,\text{d}_z w_v(b) &= 0,
\end{align*}
where $v$ is the velocity of the mixture, $\alpha$ is the vapor fraction, $w_v = \frac{\rho_v}{\alpha\rho_v + (1-\alpha)\rho_l}$ is the mass fraction of the vapor, $\rho_l$ is the mass density of the liquid, and $\rho_v$ is the mass density of the vapor. The inflow port of the pipe is located at $z=a$ where the vapor fraction and mixture velocity are known. The outflow port of the pipe is located at $z=b$ where the diffusive drift flux is zero.

- Text... 

- Text... 

## [Objectives](#toc)<a id="obj"></a>

- Text...

- Text...

## [Approach](#toc)<a id="app"></a>

The Galerkin weak formulation of the [strong form](#problem) is as follows. Find $v \in H^1\!\bigl([a,b]\bigr)$ and $\alpha \in H^1\!\bigl([a,b]\bigr)$
so that 

\begin{align*}
 \int\limits_a^b \Bigl(\alpha'\,(\rho_v-\rho_l)\,v + \bigl(\alpha\,\rho_v + (1-\alpha)\,\rho_l\bigr)\, v'\Bigr)\, \theta\, dz &= 0 \quad \forall \quad \theta \in H^1_0\!\bigl([a,b]\bigr), \text{and}
 \\
  \int\limits_a^b \rho_v\,(\alpha'\,v + \alpha\,v')\,\theta\,dz - \int\limits_a^b \frac{\rho_l\,\alpha\,D\,\rho_v\,\alpha'\,(\rho_v-\rho_l)}{\big(\alpha\,\rho_v + (1-\alpha)\,\rho_l\bigr)^2}\,\theta\,dz - \int\limits_a^b S_v\,\theta\,dz &= 0 \quad \forall \quad v \in H^1_0\!\bigl([a,b]\bigr),
\end{align*}

where $H^1\!\bigl([a,b]\bigr) := \bigl\{ u:[a,b]\subset\Reals\rightarrow \Reals \mid \int_a^b u'^2\,dx < \infty\bigr\}$ and $H^1_0\!\bigl([a,b]\bigr) := \bigl\{ w \mid w \in H^1(a,b), w(a) = 0 \bigr\}$. Both function sets as just defined are Hilbert spaces.
 
The finite element implementation of this weak form is carried out within the MOOSE framework.

- Text...

## [Results and Analysis](#toc)<a id="ranalysis"></a>

- Text...

- Text...

## [Conclusions](#toc)<a id="conclusions"></a>

- Text...

- Text...

## [References](#toc)<a id="refs"></a>

- [1] V. F. de Almeida, [*Engy-5310: Computational Continuum Transport Phenomena*](https://github.com/dpploy/engy-5310), University of Massachusetts Lowell, Dept. of Chemical Engineering (Nuclear Energy Program).
- [2] Multiphysics Object-Oriented Simulation Environment [(MOOSE)](https://mooseframework.org)
- [3] author, [reference]()
- [4] author, [reference]()